## 实例分析

- 对于某服务，`GET https://jsonplaceholder.typicode.com/users/<user-id>/posts` 将返回该用户（由`user-id`指定）发布的文章
- 系统里已有若干个用户，id为1开始的连续整数，直至返回空为止。

为了满足需求，需要将返回列表展平。

API原始数据：

- 用户1：
  - 文章1
  - 文章2
  - 文章3
  - ……
  - 文章n
- 用户2
  - 文章n+1
  - 文章n+2
  - ……
  - 文章2n

期望的迭代结果：
文章1，文章2，…，文章n，文章n+1，…


接下来我们将利用生成器实现这个需求，并利用它筛选出标题长于5个词的文章。


下面是访问服务器数据的辅助函数，不用修改

In [2]:
from requests import get


def get_posts_of(user_id):
  with get(f'https://jsonplaceholder.typicode.com/users/{user_id}/posts') as resp:
    return resp.json()


print(get_posts_of(3))

[{'userId': 3, 'id': 21, 'title': 'asperiores ea ipsam voluptatibus modi minima quia sint', 'body': 'repellat aliquid praesentium dolorem quo\nsed totam minus non itaque\nnihil labore molestiae sunt dolor eveniet hic recusandae veniam\ntempora et tenetur expedita sunt'}, {'userId': 3, 'id': 22, 'title': 'dolor sint quo a velit explicabo quia nam', 'body': 'eos qui et ipsum ipsam suscipit aut\nsed omnis non odio\nexpedita earum mollitia molestiae aut atque rem suscipit\nnam impedit esse'}, {'userId': 3, 'id': 23, 'title': 'maxime id vitae nihil numquam', 'body': 'veritatis unde neque eligendi\nquae quod architecto quo neque vitae\nest illo sit tempora doloremque fugit quod\net et vel beatae sequi ullam sed tenetur perspiciatis'}, {'userId': 3, 'id': 24, 'title': 'autem hic labore sunt dolores incidunt', 'body': 'enim et ex nulla\nomnis voluptas quia qui\nvoluptatem consequatur numquam aliquam sunt\ntotam recusandae id dignissimos aut sed asperiores deserunt'}, {'userId': 3, 'id': 25, 't

### 从结果出发

设计生成器时，从结果出发较为方便，先确定`yield`语句应该怎么写。

按需求，每次迭代时返回一个文章数据，因此`yield`应该返回一个文章结构体。


```python
def all_posts():
  # for each user ...:
  #   if ...:
  #     stop iteration...
  #
  #   for each post ...:
        yield post
```


### 先写内层

复杂的生成器一般由多层循环组成，如本项目中至少有两层（user-id循环、每个user的posts循环）。此时可以由最内层开始实现。外层循环省略，只取一个数据。

In [5]:
def all_posts():
  user_id = 3
  for post in get_posts_of(user_id):
    yield post


for post in all_posts():
  print(post)

{'userId': 3, 'id': 21, 'title': 'asperiores ea ipsam voluptatibus modi minima quia sint', 'body': 'repellat aliquid praesentium dolorem quo\nsed totam minus non itaque\nnihil labore molestiae sunt dolor eveniet hic recusandae veniam\ntempora et tenetur expedita sunt'}
{'userId': 3, 'id': 22, 'title': 'dolor sint quo a velit explicabo quia nam', 'body': 'eos qui et ipsum ipsam suscipit aut\nsed omnis non odio\nexpedita earum mollitia molestiae aut atque rem suscipit\nnam impedit esse'}
{'userId': 3, 'id': 23, 'title': 'maxime id vitae nihil numquam', 'body': 'veritatis unde neque eligendi\nquae quod architecto quo neque vitae\nest illo sit tempora doloremque fugit quod\net et vel beatae sequi ullam sed tenetur perspiciatis'}
{'userId': 3, 'id': 24, 'title': 'autem hic labore sunt dolores incidunt', 'body': 'enim et ex nulla\nomnis voluptas quia qui\nvoluptatem consequatur numquam aliquam sunt\ntotam recusandae id dignissimos aut sed asperiores deserunt'}
{'userId': 3, 'id': 25, 'title'

### 实现外层和控制



In [10]:
def all_posts():
  user_id = 1
  while True:
    post_list = get_posts_of(user_id)
    if not post_list:
      return

    for post in post_list:
      yield post
    user_id += 1


# 数据较多，下面只取前三个展示
for post in zip(range(3), all_posts()):
  print(post)

(0, {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'})
(1, {'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'})
(2, {'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'})


### 练习巩固

为了加深对设计方法的理解，请将上面代码隐藏后再实现一次

In [ ]:
def my_all_posts():
  user_id = 1
  # ...


for post in zip(range(3), my_all_posts()):
  print(post)

### 使用生成器

使用前面实现的生成器，筛选出标题`title`大于5个单词的文章。

1. 实现`is_title_longer_than_five_words`过滤函数，输入一个post结构，输出一个布尔值表示其`title`是否长于五个词
2. 使用`filter`，组合生成器和过滤函数

In [ ]:
def is_title_longer_than_five_words(post):
  return False


print(
  list(
    filter(
      is_title_longer_than_five_words,
      ...
    )
  )
)

### 参考实现

In [12]:
def is_title_longer_than_five_words(post):
  return len(post['title'].split()) > 5


print(
  list(
    filter(
      is_title_longer_than_five_words,
      all_posts()
    )
  )
)

[{'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}, {'userId': 1, 'id': 3, 'title': 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 'body': 'et iusto sed quo iure\nvoluptatem occaecati omnis eligendi aut ad\nvoluptatem doloribus vel accusantium quis pariatur\nmolestiae porro eius odio et labore et velit aut'}, {'userId': 1, 'id': 6, 'title': 'dolorem eum magni eos aperiam quia', 'body': 'ut aspernatur corporis harum nihil quis provident sequi\nmollitia nobis aliquid molestiae\nperspiciatis et ea nemo ab reprehenderit accusantium quas\nvoluptate dolores velit et doloremque molestiae'}, {'userId': 1, 'id': 9, 'title': 'nesciunt iure omnis dolorem tempora et accusantium', 'body': 'consectetur animi nesciunt iure dolore\nenim quia ad\nveniam aute